In [1]:
import numpy as np
import matplotlib.pyplot as plt
from pysrim import TRIM, SR, Ion, Element, Layer, Target
from pysrim.output import Results

In [ ]:
# Constants
micron_to_angstrom = 1e4
mm_to_angstrom = 1e7

# Layers

# 1. Al Shield (1 mm)
shield_layer = Layer(
    elements={'Al': {'stoich': 1.0}},
    density=2.70,
    width=1.0 * mm_to_angstrom  # 1 mm = 1e7 Å
)

# 2. DUT (Si)
dut_layer = Layer(
    elements={'Si': {'stoich': 1.0}},
    density=2.33,
    width=10.0 * mm_to_angstrom  # 1 μm = 1e4 Å
)

# Include or exclude the shield depending on experiment
layers = [shield_layer, dut_layer]

widths = [layer.width for layer in layers]
offsets = np.concatenate(([0], np.cumsum(widths)))

# Create the full target
target = Target(layers=layers)

In [ ]:
ion = Ion('H', energy=45.0e6)

# Initialize a TRIM calculation with given target and ion for 25 ions, quick calculation
trim = TRIM(target, ion, number_ions=500, calculation=2)

# Takes about 10 seconds on my laptop
results = trim.run()

In [ ]:
results = Results("/Users/mdovale/Applications/srim")
output_directory = '/Users/mdovale/Applications/srim_data'
TRIM.copy_output_files('/Users/mdovale/Applications/srim', output_directory)

In [ ]:
import pandas as pd

df = pd.read_csv(output_directory+'/IONIZ.txt', 
                 skiprows=40, 
                 sep='  ', 
                 names=['depth', 'ions', 'recoils'], 
                 engine='python')

df

In [ ]:
ax = df.plot(x='depth', y='ions')
ax.vlines(offsets, ymin=0, ymax=1, color='k', ls='--')

box = ax.get_position()
ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
ax.set_yscale('log')
# ax.set_xscale('log')
# ax.set_xlim(1e8,1e9)
plt.grid(which='both')
plt.tight_layout()
plt.savefig('00.pdf')
plt.show()
